In [222]:
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.dummy import DummyClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
import seaborn as sns
import matplotlib.pyplot as plt
import re
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report
from sklearn.naive_bayes import GaussianNB
from nltk.stem import WordNetLemmatizer 
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

In [218]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')
lemmatizer = WordNetLemmatizer()
values_to_remove = ['no', 'nor', 'not',"don't", "should've",  'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]
stop = [x for x in stop if x not in values_to_remove]
print(stop)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', '

[nltk_data] Downloading package stopwords to /Users/apple/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [219]:
#функция для удаления стоп слов и лемматизации
def replace_stop(string):
    string = str.lower(string)
    mas = string.split()
    for i in stop:
        while i in mas :
            mas.remove(i)
    mas = list(map(lemmatizer.lemmatize,mas))
    string  = ' '.join(mas)
    return string

#функция для очистки отзыва от знаков припенания и приведение к нижнему регистру
def clean_text(data):
    patt = re.compile("[^\w\s]")
    
    data.loc[:, "text"] = data["text"].str.replace(patt, " ", regex=True)
    data['text'] = data['text'].apply(replace_stop)
    
    return data

# Загрузка данных train 

Загружаем данные из папки train в таблицу(столбцы - отзыв, рейтинг, оценка отзыва(neg/pos))

In [220]:
train  = pd.DataFrame(columns = ['text', 'rating', 'label'])
#путь к папке train
data_path = "/Users/apple/Downloads/aclImdb/train"
pos = os.listdir(data_path + '/pos')
neg = os.listdir(data_path + '/neg')

for file in pos:
    with open(data_path + '/pos/'+ file) as f:
        contents = f.read()
    
    train.loc[len(train.index )] = [contents, int(file.split('_')[-1].split('.')[0]), 1]

for file in neg:
    with open(data_path + '/neg/'+ file) as f:
        contents = f.read()
    
    train.loc[len(train.index )] = [contents, int(file.split('_')[-1].split('.')[0]), 0]
   
    
clean_text(train)

,text,rating,label
0,movie get no respect sure lot memorable quote ...,9,1
1,bizarre horror movie filled famous face stolen...,8,1
2,solid unremarkable film matthau einstein wonde...,7,1
3,strange feeling sit alone theater occupied par...,8,1
4,probably already know 5 additional episode nev...,10,1
...,...,...,...
24995,comment may bit spoiler worth stop care enough...,3,0
24996,saucy misadventure four au pair arrive london ...,4,0
24997,oh italian assuming movie aristocrat weird fet...,1,0
24998,eight academy nomination beyond belief think b...,3,0


# Загрузка test

In [221]:
test  = pd.DataFrame(columns = ['text', 'rating', 'label'])
#путь к папке test
data_path = "/Users/apple/Downloads/aclImdb/test"
pos = os.listdir(data_path + '/pos')
neg = os.listdir(data_path + '/neg')

for file in pos:
    with open(data_path + '/pos/'+ file) as f:
        contents = f.read()
    
    test.loc[len(test.index )] = [contents, int(file.split('_')[-1].split('.')[0]), 1]

for file in neg:
    with open(data_path + '/neg/'+ file) as f:
        contents = f.read()
    
    test.loc[len(test.index )] = [contents, int(file.split('_')[-1].split('.')[0]), 0]
   
    
clean_text(test)

,text,rating,label
0,based actual story john boorman show struggle ...,9,1
1,gem film four production anticipated quality i...,9,1
2,really like show drama romance comedy rolled o...,9,1
3,best 3 experience disney themeparks certainly ...,10,1
4,korean movie seen three really stuck first exc...,10,1
...,...,...,...
24995,actor like depardieu richard really hard task ...,1,0
24996,like get couple fleeting glimpse cleavage attr...,1,0
24997,something anything want mean bound register so...,1,0
24998,heard good thing state grace came open mind th...,3,0


# Label

In [224]:
X_train, y_train = train["text"],train["label"]
X_test, y_test = test['text'], test['label']
# Векторизация текста
vectorizer = TfidfVectorizer(max_features=5000)  
X_train = vectorizer.fit_transform(X_train).toarray()
X_test = vectorizer.transform(X_test).toarray()

In [225]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)

test_accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test,y_pred,target_names=['Positive','Negative'])

print("Показатели тестовой модели")
print("Точность предсказания:", test_accuracy)
print(report)

Показатели тестовой модели
Точность предсказания: 0.84344
              precision    recall  f1-score   support

    Positive       0.83      0.86      0.85     12500
    Negative       0.86      0.83      0.84     12500

    accuracy                           0.84     25000
   macro avg       0.84      0.84      0.84     25000
weighted avg       0.84      0.84      0.84     25000



# Rating

In [226]:
test['pred'] = y_pred
test

,text,rating,label,pred
0,based actual story john boorman show struggle ...,9,1,1
1,gem film four production anticipated quality i...,9,1,1
2,really like show drama romance comedy rolled o...,9,1,1
3,best 3 experience disney themeparks certainly ...,10,1,1
4,korean movie seen three really stuck first exc...,10,1,1
...,...,...,...,...
24995,actor like depardieu richard really hard task ...,1,0,0
24996,like get couple fleeting glimpse cleavage attr...,1,0,0
24997,something anything want mean bound register so...,1,0,0
24998,heard good thing state grace came open mind th...,3,0,0


In [256]:
X_train, y_train = train["text"],train["rating"]
X_test, y_test = test['text'], test['rating']

vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)  
X_train = vectorizer.fit_transform(X_train).toarray()
X_test = vectorizer.transform(X_test).toarray()

In [257]:
X_train = np.c_[X_train, train['label']]
X_test = np.c_[X_test, test['pred']]

In [258]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)

test_accuracy = accuracy_score(y_test, y_pred)

print("Показатели тестовой модели")
print("Точность предсказания:", test_accuracy)

Показатели тестовой модели
Точность предсказания: 0.38696
